# minimal example of ontram implementation
- anlogous to https://github.com/liherz/ontram_pytorch.git

In [1]:
# Load dependencies
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import os
from sklearn.datasets import load_wine
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn 
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

from utils.graph import *

In [2]:
experiment_name = "ordinal_example"   ## <--- set experiment name
seed=42
np.random.seed(seed)

LOG_DIR="/home/bule/TramDag/dev_experiment_logs"
EXPERIMENT_DIR = os.path.join(LOG_DIR, experiment_name)
DATA_PATH = EXPERIMENT_DIR # <----------- change to different source if needed
CONF_DICT_PATH = os.path.join(EXPERIMENT_DIR, f"configuration.json")

In [3]:
# Load the dataset
wine = load_wine()
wine

{'data': array([[1.423e+01, 1.710e+00, 2.430e+00, ..., 1.040e+00, 3.920e+00,
         1.065e+03],
        [1.320e+01, 1.780e+00, 2.140e+00, ..., 1.050e+00, 3.400e+00,
         1.050e+03],
        [1.316e+01, 2.360e+00, 2.670e+00, ..., 1.030e+00, 3.170e+00,
         1.185e+03],
        ...,
        [1.327e+01, 4.280e+00, 2.260e+00, ..., 5.900e-01, 1.560e+00,
         8.350e+02],
        [1.317e+01, 2.590e+00, 2.370e+00, ..., 6.000e-01, 1.620e+00,
         8.400e+02],
        [1.413e+01, 4.100e+00, 2.740e+00, ..., 6.100e-01, 1.600e+00,
         5.600e+02]]),
 'target': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [4]:
df=pd.DataFrame(wine['data'], columns=wine['feature_names'])
df['target']=wine['target']

In [5]:
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 142 entries, 158 to 102
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   alcohol                       142 non-null    float64
 1   malic_acid                    142 non-null    float64
 2   ash                           142 non-null    float64
 3   alcalinity_of_ash             142 non-null    float64
 4   magnesium                     142 non-null    float64
 5   total_phenols                 142 non-null    float64
 6   flavanoids                    142 non-null    float64
 7   nonflavanoid_phenols          142 non-null    float64
 8   proanthocyanins               142 non-null    float64
 9   color_intensity               142 non-null    float64
 10  hue                           142 non-null    float64
 11  od280/od315_of_diluted_wines  142 non-null    float64
 12  proline                       142 non-null    float64
 13  target  

In [7]:
data_type={key:value for key, value in zip(train_df.columns, ['cont']*13+['ord'])}
data_type

{'alcohol': 'cont',
 'malic_acid': 'cont',
 'ash': 'cont',
 'alcalinity_of_ash': 'cont',
 'magnesium': 'cont',
 'total_phenols': 'cont',
 'flavanoids': 'cont',
 'nonflavanoid_phenols': 'cont',
 'proanthocyanins': 'cont',
 'color_intensity': 'cont',
 'hue': 'cont',
 'od280/od315_of_diluted_wines': 'cont',
 'proline': 'cont',
 'target': 'ord'}

In [8]:
configuration_dict=new_conf_dict(experiment_name,EXPERIMENT_DIR,DATA_PATH,LOG_DIR)
configuration_dict

{'date_of_creation': '2025-07-15 06:47:07',
 'experiment_name': 'ordinal_example',
 'PATHS': {'DATA_PATH': '/home/bule/TramDag/dev_experiment_logs/ordinal_example',
  'LOG_DIR': '/home/bule/TramDag/dev_experiment_logs',
  'EXPERIMENT_DIR': '/home/bule/TramDag/dev_experiment_logs/ordinal_example'},
 'data_type': None,
 'adj_matrix': None,
 'model_names': None,
 'seed': None,
 'nodes': None}

In [ ]:
data_type={'x1':'cont','x2':'cont','target':'ord'}  # continous , images , ordinal

levels_dict={'target':len(np.unique(df['target']))}  

adj_matrix = np.array([
    [ "0", "0", "ls"],  
    [ "0", "0", "ci"],  
    [ "0", "0", "0"],  
], object)

nn_names_matrix= create_nn_model_names(adj_matrix,data_type)
nn_names_matrix

*************
 Model has Complex intercepts and Complex shifts, please add your Model to the modelzoo 
*************


array([['0', '0', 'LinearShift'],
       ['0', '0', 'ComplexInterceptDefaultTabular'],
       ['0', '0', '0']], dtype=object)

In [36]:
quantiles = train_df.quantile([0.05, 0.95])
min_vals = quantiles.loc[0.05]
max_vals = quantiles.loc[0.95]


In [42]:
def create_node_dict_v2(adj_matrix, nn_names_matrix, data_type, min_vals, max_vals,levels_dict=None):
    """
    Creates a configuration dictionary for TRAMADAG based on an adjacency matrix,
    a neural network names matrix, and a data type dictionary.
    """
    if not validate_adj_matrix(adj_matrix):
        raise ValueError("Invalid adjacency matrix. Please check the criteria.")
    
    if len(data_type) != adj_matrix.shape[0]:
        raise ValueError("Data type dictionary should have the same length as the adjacency matrix.")
    
    target_nodes = {}
    G, edge_labels = create_nx_graph(adj_matrix, node_labels=list(data_type.keys()))
    
    sources = [node for node in G.nodes if G.in_degree(node) == 0]
    sinks = [node for node in G.nodes if G.out_degree(node) == 0]
    
    for i, node in enumerate(G.nodes):
        parents = list(G.predecessors(node))
        target_nodes[node] = {}
        target_nodes[node]['Modelnr'] = i
        target_nodes[node]['data_type'] = data_type[node]
        
        # write the levels of the ordinal outcome
        if data_type[node]=='ord':
            if levels_dict is None:
                print('provide levels_dict e.g. {"x3":3}')
            else:
                target_nodes[node]['levels'] = levels_dict[node]
            
    
        target_nodes[node]['node_type'] = "source" if node in sources else "sink" if node in sinks else "internal"
        target_nodes[node]['parents'] = parents
        target_nodes[node]['parents_datatype'] = {parent:data_type[parent] for parent in parents}
        target_nodes[node]['transformation_terms_in_h()'] = {parent: edge_labels[(parent, node)] for parent in parents if (parent, node) in edge_labels}
        target_nodes[node]['min'] = min_vals.iloc[i].tolist()   
        target_nodes[node]['max'] = max_vals.iloc[i].tolist()

        
        transformation_term_nn_models = {}
        for parent in parents:
            parent_idx = list(data_type.keys()).index(parent)  
            child_idx = list(data_type.keys()).index(node) 
            
            if nn_names_matrix[parent_idx, child_idx] != "0":
                transformation_term_nn_models[parent] = nn_names_matrix[parent_idx, child_idx]
        target_nodes[node]['transformation_term_nn_models_in_h()'] = transformation_term_nn_models
    return target_nodes

In [43]:
target_nodes=create_node_dict_v2(adj_matrix, nn_names_matrix, data_type, min_vals, max_vals,levels_dict)

In [39]:
from utils.tram_model_helpers import *
from utils.tram_models import *

In [44]:
# TODO : Where a SimpleIntercept model is there must be C-1 thetas 


def get_fully_specified_tram_model_v2(node: str, target_nodes: dict, verbose=True):
    # Source nodes get a simple intercept only
    if target_nodes[node]['node_type'] == 'source':
        # if target node is ordinal we only need c-1 thetas
        if target_nodes[node]['data_type']=='ord':
            nn_int = SimpleIntercept(n_thetas=target_nodes[node]['levels']-1)
        else:    
            nn_int = SimpleIntercept()
            
        model = TramModel(nn_int, None)
        if verbose:
            print("Source → SimpleIntercept only")
        return model

    # Otherwise gather terms and model names
    _, terms_dict, model_names_dict = ordered_parents(node, target_nodes)
    model_dict = merge_transformation_dicts(terms_dict, model_names_dict)

    # Split intercepts vs. shifts
    intercepts_dict = {
        k: v for k, v in model_dict.items()
        if "ci" in v['h_term'] or "si" in v['h_term']
    }
    shifts_dict = {
        k: v for k, v in model_dict.items()
        if "ci" not in v['h_term'] and "si" not in v['h_term']
    }

    # Build intercept network
    intercept_groups = group_by_base(intercepts_dict, prefixes=("ci", "si"))
    if not intercept_groups:

        if target_nodes[node]['data_type']=='ord':
            nn_int = SimpleIntercept(n_thetas=int(target_nodes[node]['levels'])-1)
        else:    
            nn_int = SimpleIntercept()

    else:
        if len(intercept_groups) > 1:
            raise ValueError("Multiple intercept models detected; only one is supported.")
        feats = next(iter(intercept_groups.values()))
        cls_name = feats[0][1]['class_name']
        base = get_base_model_class(cls_name)
        
        
        if target_nodes[node]['data_type']=='ord':
            nn_int = globals()[base](n_features=len(feats),n_thetas=int(target_nodes[node]['levels'])-1)
        else:    
            nn_int = globals()[base](n_features=len(feats))
        

    # Build shift networks (handles both "cs" and "ls")
    shift_groups = group_by_base(shifts_dict, prefixes=("cs", "ls"))
    nn_shifts = []
    for feats in shift_groups.values():
        cls_name = feats[0][1]['class_name']
        base = get_base_model_class(cls_name)
        nn_shifts.append(globals()[base](n_features=len(feats)))

    # Combine into TramModel
    tram_model = TramModel(nn_int, nn_shifts)
    if verbose:
        print("Constructed TRAM model:", tram_model)
    return tram_model

In [45]:
get_fully_specified_tram_model_v2('target', target_nodes, verbose=True)

Constructed TRAM model: TramModel(
  (nn_int): ComplexInterceptDefaultTabular(
    (fc1): Linear(in_features=1, out_features=8, bias=True)
    (relu1): ReLU()
    (fc2): Linear(in_features=8, out_features=8, bias=True)
    (relu2): ReLU()
    (fc3): Linear(in_features=8, out_features=2, bias=False)
  )
  (nn_shift): ModuleList(
    (0): LinearShift(
      (fc): Linear(in_features=1, out_features=1, bias=False)
    )
  )
)


TramModel(
  (nn_int): ComplexInterceptDefaultTabular(
    (fc1): Linear(in_features=1, out_features=8, bias=True)
    (relu1): ReLU()
    (fc2): Linear(in_features=8, out_features=8, bias=True)
    (relu2): ReLU()
    (fc3): Linear(in_features=8, out_features=2, bias=False)
  )
  (nn_shift): ModuleList(
    (0): LinearShift(
      (fc): Linear(in_features=1, out_features=1, bias=False)
    )
  )
)